In [1]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import glob, os
import numpy as np
import gc
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import Sequence
import random
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [ ]:
x_real = np.load('C:/Users/h4uh4/OneDrive/Desktop/Skripsi_Bimbingan/Fingerprint/DataFinal/SOCOFing/dataset/x_easy.npz')['data']
x_easy = np.load('C:/Users/h4uh4/OneDrive/Desktop/Skripsi_Bimbingan/Fingerprint/DataFinal/SOCOFing/dataset/x_easy.npz')['data']
x_medium = np.load('C:/Users/h4uh4/OneDrive/Desktop/Skripsi_Bimbingan/Fingerprint/DataFinal/SOCOFing/dataset/x_medium.npz')['data']
x_hard = np.load('C:/Users/h4uh4/OneDrive/Desktop/Skripsi_Bimbingan/Fingerprint/DataFinal/SOCOFing/dataset/x_hard.npz')['data']
y_real = np.load('C:/Users/h4uh4/OneDrive/Desktop/Skripsi_Bimbingan/Fingerprint/DataFinal/SOCOFing/dataset/y_real.npy')
y_easy = np.load('C:/Users/h4uh4/OneDrive/Desktop/Skripsi_Bimbingan/Fingerprint/DataFinal/SOCOFing/dataset/y_easy.npy')
y_medium = np.load('C:/Users/h4uh4/OneDrive/Desktop/Skripsi_Bimbingan/Fingerprint/DataFinal/SOCOFing/dataset/y_medium.npy')
y_hard = np.load('C:/Users/h4uh4/OneDrive/Desktop/Skripsi_Bimbingan/Fingerprint/DataFinal/SOCOFing/dataset/y_hard.npy')

print(x_real.shape, y_real.shape)

plt.figure(figsize=(15, 10))
plt.subplot(1, 4, 1)

plt.title(y_real[0])
plt.imshow(x_real[0].squeeze(), cmap='gray')
plt.subplot(1, 4, 2)

plt.title(y_easy[0])
plt.imshow(x_easy[0].squeeze(), cmap='gray')
plt.subplot(1, 4, 3)

plt.title(y_medium[0])
plt.imshow(x_medium[0].squeeze(), cmap='gray')
plt.subplot(1, 4, 4)

plt.title(y_hard[0])
plt.imshow(x_hard[0].squeeze(), cmap='gray')


In [ ]:
import cv2
import matplotlib.pyplot as plt
import albumentations as A


image = cv2.imread('C:/Users/h4uh4/OneDrive/Desktop/Skripsi_Bimbingan/Fingerprint/DataFinal/SOCOFing/Real/1__M_Left_index_finger.BMP')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

augment = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Affine(scale=(0.9, 1.1), translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)}, rotate=(-5, 5), p=0.7),



])






augmented = augment(image=image)
augmented_image = augmented['image']


plt.figure(figsize=(10, 5))


plt.subplot(1, 2, 1)
plt.title("Original Image")
plt.imshow(image)
plt.axis('off')


plt.subplot(1, 2, 2)
plt.title("Augmented Image")
plt.imshow(augmented_image)
plt.axis('off')

plt.show()


In [ ]:

x_real = x_real[:len(y_real)]


x_data = np.concatenate([x_real, x_easy, x_medium, x_hard], axis=0)
y_data = np.concatenate([y_real, y_easy, y_medium, y_hard], axis=0)

print(f"x_data shape: {x_data.shape}")
print(f"y_data shape: {y_data.shape}")


half_size = len(x_data) // 2

x_data_half = x_data[:half_size]
y_data_half = y_data[:half_size]

print(f"x_data_half shape: {x_data_half.shape}")
print(f"y_data_half shape: {y_data_half.shape}")



assert x_data_half.shape[0] == y_data_half.shape[0], "Jumlah sample x_data dan y_data tidak sama!"


x_temp, x_test, y_temp, y_test = train_test_split(x_data_half, y_data_half, test_size=0.2, random_state=42)


x_train, x_val, y_train, y_val = train_test_split(x_temp, y_temp, test_size=0.25, random_state=42)


print(f"Train set size: {x_train.shape[0]} samples")
print(f"Validation set size: {x_val.shape[0]} samples")
print(f"Test set size: {x_test.shape[0]} samples")


In [ ]:

print("Memeriksa dataset raw untuk NaN atau Inf...")

if np.any(np.isnan(x_data_half)) or np.any(np.isinf(x_data_half)):
    print("NaN atau Inf terdeteksi pada dataset raw!")
else:
    print("Dataset raw bersih dari NaN atau Inf.")


In [ ]:

x_train = np.expand_dims(x_train, -1).astype('float32') / 255.
x_val = np.expand_dims(x_val, -1).astype('float32') / 255.
x_test = np.expand_dims(x_test, -1).astype('float32') / 255.


In [ ]:
import cv2
import numpy as np
import psutil
import matplotlib.pyplot as plt


def apply_gabor_filter(img):
    ksize = 15
    sigma = 0.75
    theta = np.pi / 4
    lambda_ = 5.0
    gamma = 0.5
    psi = 0


    gabor_kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambda_, gamma, psi)


    filtered_image = cv2.filter2D(img, cv2.CV_32F, gabor_kernel)


    filtered_image = np.clip(filtered_image, 0, 255)
    filtered_image = filtered_image / 255.0

    return filtered_image





x_train_gabor = np.array([apply_gabor_filter(img.squeeze()) for img in x_train])
x_val_gabor = np.array([apply_gabor_filter(img.squeeze()) for img in x_val])
x_test_gabor = np.array([apply_gabor_filter(img.squeeze()) for img in x_test])


print(f"Memory usage: {psutil.virtual_memory().percent}%")


plt.figure(figsize=(10, 5))


plt.subplot(1, 3, 1)
plt.imshow(x_train[0], cmap='gray')
plt.title("Original Image")
plt.axis('off')


plt.subplot(1, 3, 2)
plt.imshow(x_train_gabor[0], cmap='gray')
plt.title("Gabor Filtered Image")
plt.axis('off')


plt.subplot(1, 3, 3)
gabor_result = apply_gabor_filter(x_train[0])
plt.imshow(gabor_result, cmap='gray')
plt.title("Gabor Filtered Image")
plt.axis('off')

plt.show()


In [ ]:
import gc


gc.collect()


In [ ]:

x_train_gabor = np.nan_to_num(x_train_gabor)
x_val_gabor = np.nan_to_num(x_val_gabor)
x_test_gabor = np.nan_to_num(x_test_gabor)


x_train_gabor = np.expand_dims(x_train_gabor, -1).astype('float32') / np.max(x_train_gabor)
x_val_gabor = np.expand_dims(x_val_gabor, -1).astype('float32') / np.max(x_val_gabor)
x_test_gabor = np.expand_dims(x_test_gabor, -1).astype('float32') / np.max(x_test_gabor)


In [ ]:
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras.utils import Sequence


def ensure_rgb(image):
    if image.shape[-1] == 1:
        image = np.repeat(image, 3, axis=-1)
    return image


x_train_gabor_rgb = np.array([ensure_rgb(img) for img in x_train_gabor])


class PairGenerator(Sequence):
    def __init__(self, x, y, x_real, y_real, batch_size=32, augment_fn=None):
        self.x = x
        self.y = y
        self.x_real = x_real
        self.y_real = y_real
        self.batch_size = batch_size
        self.augment_fn = augment_fn


        self.label_real_dict = {}
        for idx, label in enumerate(self.y_real):
            key = ''.join(label.astype(str)).zfill(6)
            if key not in self.label_real_dict:
                self.label_real_dict[key] = []
            self.label_real_dict[key].append(idx)

        self.label_keys = list(self.label_real_dict.keys())

    def __len__(self):
        return int(np.ceil(len(self.x) / self.batch_size))

    def __getitem__(self, index):
        start = index * self.batch_size
        end = min((index + 1) * self.batch_size, len(self.x))


        x1_batch = np.empty((end - start, 96, 96, 3), dtype=np.float32)
        x2_batch = np.empty((end - start, 96, 96, 3), dtype=np.float32)
        y_batch = self.y[start:end]
        batch_size = len(x1_batch)

        labels = np.zeros((batch_size, 1), dtype=np.float32)

        for i, label in enumerate(y_batch):
            key = ''.join(label.astype(str)).zfill(6)

            if random.random() > 0.5 and key in self.label_real_dict:
                idx = random.choice(self.label_real_dict[key])
                x2 = self.x_real[idx]
                labels[i] = 1.
            else:
                while True:
                    unmatch_key = random.choice(self.label_keys)
                    if unmatch_key != key:
                        idx = random.choice(self.label_real_dict[unmatch_key])
                        x2 = self.x_real[idx]
                        break
                labels[i] = 0.

            x2_batch[i] = x2


            x1_batch[i] = self.x[start + i]
            x2_batch[i] = x2


            if self.augment_fn:

                if isinstance(x1_batch[i], np.ndarray):
                    x1_batch[i] = self.augment_fn(image=x1_batch[i])['image']
                elif isinstance(x1_batch[i], tf.Tensor):
                    x1_batch[i] = self.augment_fn(image=x1_batch[i].numpy())['image']
                
                if isinstance(x2_batch[i], np.ndarray):
                    x2_batch[i] = self.augment_fn(image=x2_batch[i])['image']
                elif isinstance(x2_batch[i], tf.Tensor):
                    x2_batch[i] = self.augment_fn(image=x2_batch[i].numpy())['image']

        return (x1_batch, x2_batch), labels


In [ ]:
import numpy as np


def ensure_rgb(image):
    if image.shape[-1] == 1:
        image = np.repeat(image, 3, axis=-1)
    return image


x_train_gabor_rgb = np.array([ensure_rgb(img) for img in x_train_gabor])


print("Shape of first image after RGB conversion:", x_train_gabor_rgb[0].shape)


import matplotlib.pyplot as plt
plt.imshow(x_train_gabor_rgb[0])
plt.title('First RGB Image')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt


for i in range(5):
    print(f"Image {i} shape:", x_train_gabor_rgb[i].shape)


for i in range(5):
    print(f"Image {i} min value:", np.min(x_train_gabor_rgb[i]))
    print(f"Image {i} max value:", np.max(x_train_gabor_rgb[i]))


for i in range(5):
    plt.figure(figsize=(10, 5))
    
    plt.subplot(1, 2, 1)
    if x_train_gabor[i].shape[-1] == 1:
        plt.imshow(x_train_gabor[i].squeeze(), cmap='gray')
    else:
        plt.imshow(x_train_gabor[i])
    plt.title(f'Image {i} Before RGB Conversion')


    rgb_image = ensure_rgb(x_train_gabor[i])

    plt.subplot(1, 2, 2)
    plt.imshow(rgb_image)
    plt.title(f'Image {i} After RGB Conversion')

    plt.show()


In [ ]:

plt.imshow(x_train_gabor_rgb[0])
plt.title('First x1 Image')
plt.show()


In [ ]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras import layers, models


def build_vgg19_model(input_shape=(96, 96, 3)):

    base_model = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)


    for layer in base_model.layers:
        layer.trainable = False
    

    x = layers.Flatten()(base_model.output)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dense(64, activation='relu')(x)
    output = layers.Dense(1, activation='sigmoid')(x)


    model = models.Model(inputs=base_model.input, outputs=output)

    return model


x1 = layers.Input(shape=(96, 96, 3))
x2 = layers.Input(shape=(96, 96, 3))


vgg_model = build_vgg19_model()


f1 = vgg_model(x1)
f2 = vgg_model(x2)


diff = layers.Subtract()([f1, f2])


x = layers.Dense(128, activation='relu')(diff)
x = layers.Dense(64, activation='relu')(x)
output = layers.Dense(1, activation='sigmoid')(x)


model = models.Model(inputs=[x1, x2], outputs=output)


model.summary()


In [ ]:

print("Memeriksa dataset setelah filter Laplacian...")

if np.any(np.isnan(x_train_gabor_rgb)) or np.any(np.isinf(x_train_gabor_rgb)):
    print("NaN atau Inf terdeteksi pada dataset yang sudah difilter!")
else:
    print("Dataset yang sudah difilter bersih dari NaN atau Inf.")


In [ ]:
train_gen = PairGenerator(x_train_gabor_rgb, y_train, x_train_gabor_rgb, y_train, batch_size=32, augment_fn=augment)
val_gen = PairGenerator(x_val, y_val, x_val, y_val, batch_size=32, augment_fn=None)


In [ ]:

print(f"NaN atau Inf terdeteksi pada x_train_gabor setelah filter Gabor: {np.any(np.isnan(x_train_gabor_rgb)) or np.any(np.isinf(x_train_gabor_rgb))}")


print(f"NaN atau Inf terdeteksi pada x_val_gabor setelah filter Gabor: {np.any(np.isnan(x_val_gabor)) or np.any(np.isinf(x_val_gabor))}")
print(f"NaN atau Inf terdeteksi pada x_test_gabor setelah filter Gabor: {np.any(np.isnan(x_test_gabor)) or np.any(np.isinf(x_test_gabor))}")


In [ ]:

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC()])


early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint('vgg19_best.h5', save_best_only=True, monitor='val_loss')
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=0.0001)


history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    callbacks=[checkpoint, early_stop, lr_scheduler],
    verbose=2
)

In [ ]:

model.save('C:/Users/h4uh4/OneDrive/Desktop/Skripsi_Bimbingan/Fingerprint/Compare/Diagram/BiarGaBingung/VGG/Model_VGG/gabor_vgg19_final_model.keras')



In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

test_gen = PairGenerator(x_test_gabor, y_test, x_test_gabor, y_test, batch_size=32, augment_fn=None)


test_loss, test_accuracy, test_auc = model.evaluate(test_gen)


print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test AUC: {test_auc}")


results = [test_loss, test_accuracy, test_auc]
labels = ['Test Loss', 'Test Accuracy', 'Test AUC']


plt.figure(figsize=(8, 6))
plt.bar(labels, results, color=['red', 'green', 'blue'])


plt.xlabel('Metric')
plt.ylabel('Value')
plt.title('Model Evaluation Results')


plt.show()
